In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [5]:
tf.constant(1) < tf.constant(2) < tf.constant(2) or tf.constant(False)

<tf.Tensor: shape=(), dtype=bool, numpy=False>

In [2]:
tf.unstack(tf.constant([[1,2],[3,4]]))[0].shape

TensorShape([2])

In [4]:
pd.DataFrame(data=None, columns=["weight", "intersect", "xo", "yo", "xf", "yf"]).shape[1]

6

In [3]:
tf.constant(True).numpy()

True

In [8]:
num_lines = sum(1 for line in open('out.csv'))
num_lines

72314946

In [7]:
a = np.array(
  [[1, 2 ,3],
   [4, 5, 6]]
)
b = np.array(
  [1, 2]
)>1
a[b]

array([[4, 5, 6]])

In [3]:
filenames = ["out_0.tfrecord"]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [57]:
features = {
  "weight": tf.io.FixedLenFeature([], tf.string),
  "intersect": tf.io.FixedLenFeature([], tf.string),
  "full_arr": tf.io.FixedLenFeature([], tf.string)
}
def process(raw_record):
  tensor_dict = tf.io.parse_single_example(raw_record, features)

  weight = tf.io.parse_tensor(tensor_dict["weight"], out_type=tf.float32).numpy()
  intersect = tf.io.parse_tensor(tensor_dict["intersect"], out_type=tf.bool).numpy()
  full_arr = tf.io.parse_tensor(tensor_dict["full_arr"], out_type=tf.float32).numpy()
  print(np.unique(full_arr[:, 0]).shape)

  weight = np.expand_dims(np.repeat(weight, 8796), -1)
  intersect = np.expand_dims(intersect, -1)

  data = np.hstack([weight, intersect, full_arr])
  df = pd.DataFrame(data=data, columns=["weight", "intersect", "xo", "yo", "xf", "yf"])
  df = df[df["intersect"] == True]
  df.drop("intersect", axis="columns")
  df.to_csv("out.csv", mode="a", header=not os.path.exists("out.csv"), float_format="%.4f")

  return tf.constant(1.0, dtype=tf.float32)


In [58]:
features = {
  "weight": tf.io.FixedLenFeature([], tf.string),
  "intersect": tf.io.FixedLenFeature([], tf.string),
  "full_arr": tf.io.FixedLenFeature([], tf.string)
}
for raw_record in raw_dataset.take(1):
  process(raw_record)
  print(i)

(100,)
103


In [17]:
result = {}
# example.features.feature is the dictionary
for key, feature in example.features.feature.items():
  # The values are the Feature objects which contain a `kind` which contains:
  # one of three fields: bytes_list, float_list, int64_list

  print(type(feature))
  kind = feature.WhichOneof('kind')
  print(kind)
  print(type(tf.io.parse_getattr(feature, kind).value))
  result[key] = np.array(getattr(feature, kind).value)

<class 'tensorflow.core.example.feature_pb2.Feature'>
bytes_list
<class 'google.protobuf.pyext._message.RepeatedScalarContainer'>
<class 'tensorflow.core.example.feature_pb2.Feature'>
bytes_list
<class 'google.protobuf.pyext._message.RepeatedScalarContainer'>
<class 'tensorflow.core.example.feature_pb2.Feature'>
bytes_list
<class 'google.protobuf.pyext._message.RepeatedScalarContainer'>


In [11]:
result.keys()

dict_keys(['weight', 'full_arr', 'intersect'])

In [13]:
result["weight"]

array([b'\x08\x01\x12\x08\x12\x02\x08d\x12\x02\x08\x01"\x90\x03\x9f\x9a\xba=\xeb\x8c\x0e>\xb1\xe9\xb1=\xcf\xc7\xfc= \xabT>E\x9b\x1b<\x0f\xe4\xbf=a\x0b\xbd=\x1c\xfb!>\x03\x11\x00>\xba\xd8\xb9=\xe5S)=S2\x85=Y#\x01>-_\x80=\x9f\xeb\xb2=\xd7\xbbG>\xd0t\xd6=\xcb6 >\xb6\x8d\xb9=\x8f\xd2\xb1=+$\x0c>\xae\xd6\x01>\x9b\x95k=\x01\x05f>\x97o$>\x86{;>\xd9NE>\x8a"\x1a>\x8b\xfd\x8c=\xc5\x11P>\x95\x97\xc2=\x07\x02M>>K\xb2=PzE>\xda\xe4\xec=\xbf\xb1\x92=r\x0cE>\xfb\xd2+>k\xb1\r>\x1fJ\xf4=\x9a\xd9H>O\x8bq;x\x88\xf2=\xe0\x12\x0e>M.\x8d=\xbf5\xe2=\x1d\xa0\xda=G\x03\xc2=}.\xd6=\xfao\xc6=\x8cr\xf8<\xf5v\xbb=m\xa0o=\xdcT+>\xc7\xfe\x83=C\xc8\xaa=\x87\x9e,=M5W=G;\x8e=\xee\x12#>\x13u\xf5=24\xac=\xe1\xbd\xe2=/\xe3\xc0=R{\x1c>\xae\x8a\x9b=7\xd6\xd0=\x97\xa1\xfd=\x8a\x03\x0c>\xe6\x8b<>\xbd\x8aG=\x7fM\xdb=\x91\x0f\xb0=\x9a\x1a\xcb=4e1>\xd9f\xc6=\xae;\xe3=!g7>ay`=\xed\x9f\xd7=i\x941>*\x15\xc8=\x00\x92\xfb=\xbc\xe0\xb1=\x83\xf7\r>%\x96\x9c=\xb4d\xa1<\xdb\xeb\xdb=\x85\x81J>\xda\xa4\xbf;\xd8\x19\x97=\x9aW\x9e=\xa0q\x06=\

In [39]:
arr = np.array([1, 2, 3, 4])
np.repeat(arr, 2)

array([1, 1, 2, 2, 3, 3, 4, 4])